# [프로젝트] 멋진챗봇만들기

### Step 1. 데이터 다운로드
### Step 2. 데이터 정제
### Step 3. 데이터 토큰화
### Step 4. Augmentation
### Step 5. 데이터 벡터화
### Step 6. 훈련하기
### Step 7. 성능 측정하기

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

import re
import os
import random
import math

from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

print(tf.__version__)

2.6.0


## Step 1. 데이터 다운로드
ChatbotData.csv 사용, 질문(questions), 답변(answers)

In [2]:
import pandas as pd

# Chatbot.csv 파일 경로 설정 (로컬 경로를 지정해 주세요)
file_path = "Chatbot.csv"

# CSV 파일 읽기
data = pd.read_csv(file_path)

# 데이터 확인 (첫 5줄)
print(data.head())

# 질문과 답변을 각각 분리해서 저장
questions = data['Q'].tolist()  # 질문 열(Q)
answers = data['A'].tolist()    # 답변 열(A)

# 데이터 확인
print(f"총 질문 수: {len(questions)}")
print(f"총 답변 수: {len(answers)}")

# 첫 번째 질문과 답변 출력
print(f"첫 번째 질문: {questions[0]}")
print(f"첫 번째 답변: {answers[0]}")

                 Q            A  label
0           12시 땡!   하루가 또 가네요.      0
1      1지망 학교 떨어졌어    위로해 드립니다.      0
2     3박4일 놀러가고 싶다  여행은 언제나 좋죠.      0
3  3박4일 정도 놀러가고 싶다  여행은 언제나 좋죠.      0
4          PPL 심하네   눈살이 찌푸려지죠.      0
총 질문 수: 11823
총 답변 수: 11823
첫 번째 질문: 12시 땡!
첫 번째 답변: 하루가 또 가네요.


In [3]:
questions = data['Q'].tolist()  # 질문 열 이름 확인 후 사용
answers = data['A'].tolist()    # 답변 열 이름 확인 후 사용

print("questions 샘플:", questions[:5])
print("answers 샘플:", answers[:5])

questions 샘플: ['12시 땡!', '1지망 학교 떨어졌어', '3박4일 놀러가고 싶다', '3박4일 정도 놀러가고 싶다', 'PPL 심하네']
answers 샘플: ['하루가 또 가네요.', '위로해 드립니다.', '여행은 언제나 좋죠.', '여행은 언제나 좋죠.', '눈살이 찌푸려지죠.']


## Step 2. 데이터 정제
- 영문자의 경우, 모두 소문자로 변환합니다.
- 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.

In [4]:
import re

def preprocess_sentence(sentence):
    # 1. 영문자를 소문자로 변환
    sentence = sentence.lower()
    
    # 2. 영문자, 한글, 숫자, 주요 특수문자를 제외한 모든 문자를 제거
    sentence = re.sub(r"[^a-zA-Z0-9가-힣?.!,]+", " ", sentence)
    
    # 3. 문자열 양 끝의 공백 제거
    sentence = sentence.strip()
    
    return sentence

# 예시 데이터로 함수 테스트
test_sentence = "Hello! 내 이름은 김민혁입니다, 특수문자 !@#$%"
preprocessed_sentence = preprocess_sentence(test_sentence)
print("전처리 후:", preprocessed_sentence)

전처리 후: hello! 내 이름은 김민혁입니다, 특수문자 !


## Step 3. 데이터 토큰화
> 토큰화(mecab), build_corpus() 함수 구현
- 소스 문장 데이터와 타겟 문장 데이터를 입력
- 데이터를 앞서 정의한 preprocess_sentence() 함수로 정제하고, 토큰화
- 토큰화는 전달받은 토크나이즈 함수를 사용, mecab.morphs 함수를 전달
- 토큰의 개수가 일정 길이 이상인 문장은 데이터에서 제외
- 중복되는 문장은 데이터에서 제외
    - 소스 : 타겟 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사.

In [5]:
from konlpy.tag import Mecab
import re

mecab = Mecab()

def build_corpus(sentences, tokenizer, max_len=40):
    corpus = []

    seen_sentences = set()

    for sentence in sentences:
        # Step 1: 문장 정제
        sentence = preprocess_sentence(sentence)

        # Step 2: 토큰화
        tokens = tokenizer(sentence)

        # Step 3: 토큰의 개수가 max_len보다 길면 제외
        if len(tokens) > max_len:
            continue

        # Step 4: 중복 문장 제거
        tokenized_sentence = ' '.join(tokens)
        if tokenized_sentence in seen_sentences:
            continue
        seen_sentences.add(tokenized_sentence)

        # 최종 토큰화된 문장을 corpus에 추가
        corpus.append(tokens)

    return corpus

# que_corpus 와 ans_corpus 생성
que_corpus = build_corpus(questions, mecab.morphs, max_len=40)
ans_corpus = build_corpus(answers, mecab.morphs, max_len=40)


## Step 4. Augmentation
한국어로 사전 훈련된 Embedding model 다운로드 - ko.bin 파일

In [6]:
# gensim 다운그레이드
!pip install gensim==3.8.3

In [7]:
from gensim.models import Word2Vec

# ko.bin 파일 경로 설정
model_path = 'ko.bin'

# Word2Vec 모델 로드
wv = Word2Vec.load(model_path)
print("모델 로드 완료!")

# 모델 테스트 예시
print(wv.most_similar("학교"))  # "학교"와 유사한 단어 출력

모델 로드 완료!
[('학교의', 0.7560996413230896), ('강습소', 0.7425637245178223), ('중고등학교', 0.7386142015457153), ('전문학교', 0.7356827855110168), ('사립학교', 0.7347193956375122), ('소학교', 0.7305554747581482), ('여학교', 0.7091007232666016), ('사범학교', 0.6901223659515381), ('대학', 0.6897724866867065), ('학원', 0.6869212985038757)]


/tmp/ipykernel_249/691456926.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  print(wv.most_similar("학교"))  # "학교"와 유사한 단어 출력


In [8]:
wv.most_similar("바나나")

/tmp/ipykernel_249/2324208733.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  wv.most_similar("바나나")


[('코코넛', 0.8097119927406311),
 ('시금치', 0.7701147794723511),
 ('레몬', 0.76884925365448),
 ('땅콩', 0.7684735059738159),
 ('파인애플', 0.7639915347099304),
 ('녹차', 0.7631460428237915),
 ('딸기', 0.7617197036743164),
 ('바닐라', 0.7497864961624146),
 ('파슬리', 0.7447543144226074),
 ('코코아', 0.7408244609832764)]

In [9]:
import random
from tqdm import tqdm

# lexical_sub 함수 정의: 단어를 유사도가 높은 단어로 치환하여 데이터 증강
def lexical_sub(sentence, model, prob=0.7):
    words = sentence.split()
    new_words = []
    
    for word in words:
        if word in model and random.random() < prob:
            try:
                similar_words = model.most_similar(word, topn=5)
                new_word = random.choice(similar_words)[0]
                new_words.append(new_word)
            except KeyError:
                new_words.append(word)
        else:
            new_words.append(word)
    
    return ' '.join(new_words)

# 데이터 증강 수행 함수
def augment_corpus(questions, answers, model):
    augmented_questions = []
    augmented_answers = []

    for que, ans in tqdm(zip(questions, answers), total=len(questions)):
        # 1. 질문을 증강하고 답변은 원본 유지
        augmented_questions.append(lexical_sub(que, model))  # 개별 질문
        augmented_answers.append(ans)  # 개별 답변

        # 2. 답변을 증강하고 질문은 원본 유지
        augmented_questions.append(que)
        augmented_answers.append(lexical_sub(ans, model))
        
        # 3. 질문과 답변을 모두 증강
        augmented_questions.append(lexical_sub(que, model))
        augmented_answers.append(lexical_sub(ans, model))

    # 최종 데이터셋 생성
    final_questions = questions + augmented_questions
    final_answers = answers + augmented_answers

    print(f"원본 질문 데이터 개수: {len(questions)}")
    print(f"증강된 질문 데이터 개수: {len(augmented_questions)}")
    print(f"전체 질문 데이터 개수: {len(final_questions)}")

    print(f"원본 답변 데이터 개수: {len(answers)}")
    print(f"증강된 답변 데이터 개수: {len(augmented_answers)}")
    print(f"전체 답변 데이터 개수: {len(final_answers)}")

    return final_questions, final_answers

# 실제 데이터에 증강 적용
final_questions, final_answers = augment_corpus(questions, answers, wv)

  0%|          | 0/11823 [00:00<?, ?it/s]/tmp/ipykernel_249/3065843551.py:10: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  if word in model and random.random() < prob:
/tmp/ipykernel_249/3065843551.py:12: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  similar_words = model.most_similar(word, topn=5)
100%|██████████| 11823/11823 [01:06<00:00, 178.01it/s]

원본 질문 데이터 개수: 11823
증강된 질문 데이터 개수: 35469
전체 질문 데이터 개수: 47292
원본 답변 데이터 개수: 11823
증강된 답변 데이터 개수: 35469
전체 답변 데이터 개수: 47292


### 오류 🚨🚨🚨
앞에서 questions와 answers를 잘 불러왔다가 중간에 내가 

`questions=[]`
`answers =[]`

를 선언해서 정제했던 데이터를 다 날렸길래 데이터 수가 다 0으로 지정되어졌다.

그래서 데이터 증강이 실현되지 않았던 걸 확인했다.

## Step 5. 데이터 벡터화
- `<start>` 토큰과 `<end>` 토큰
- `que_corpus` 와 결합하여 전체 데이터에 대한 단어 사전을 구축
- 벡터화하여 `enc_train` 과 `dec_train` 얻기

In [10]:
# 답변 데이터에 <start>와 <end> 토큰 추가
ans_corpus_with_tokens = [["<start>"] + ans.split() + ["<end>"] for ans in final_answers]

# 결과 예시 확인
sample_data = ["12", "시", "땡", "!"]
print(["<start>"] + sample_data + ["<end>"])

# 2. que_corpus와 ans_corpus를 결합하여 단어 사전 생성
from tensorflow.keras.preprocessing.text import Tokenizer

# 모든 데이터를 단어 사전에 포함시키기 위해 que_corpus와 ans_corpus_with_tokens 결합
total_corpus = final_questions + ans_corpus_with_tokens

# 토크나이저 초기화 및 단어 사전 생성
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(total_corpus)

# 단어 사전 크기 확인
vocab_size = len(tokenizer.word_index) + 1  # 패딩 토큰 포함

print("단어 사전 크기:", vocab_size)

# 3. 각 문장을 시퀀스 번호로 변환하여 벡터화
# 질문과 답변 각각에 대해 벡터화 수행
enc_train = tokenizer.texts_to_sequences(final_questions)
dec_train = tokenizer.texts_to_sequences(ans_corpus_with_tokens)

# 4. 패딩 처리
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 최대 길이 설정
max_len = 50  # 필요에 따라 수정 가능

enc_train = pad_sequences(enc_train, maxlen=max_len, padding='post')
dec_train = pad_sequences(dec_train, maxlen=max_len, padding='post')

# 결과 확인
print(f"인코더 데이터 크기: {enc_train.shape}")
print(f"디코더 데이터 크기: {dec_train.shape}")

['<start>', '12', '시', '땡', '!', '<end>']
단어 사전 크기: 25118
인코더 데이터 크기: (47292, 50)
디코더 데이터 크기: (47292, 50)


## Step 6. 훈련하기
- Transformer 모델 설계
    - Positional Encoding
    - Mask generate
    - Multi-Head Attention
    - Position-wise Feed-Forward Network
    - Encoder Layer
    - Decoder Layer
    - Encoder
    - Decoder

Positonal Encoding

In [11]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table


mask 생성

In [12]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask


Multi-head Attention

In [13]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

Position-wise Feed Forward Network

In [14]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out


Encoder Layer

In [15]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn


Decoder Layer

In [16]:

class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn


Encoder

In [17]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns


Decoder

In [18]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns


Transformer 조립하기

In [19]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns


모델 인스턴스 생성

In [20]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)
		
d_model = 512


Learning Rate Scheduler

In [21]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)


Learning Rate & Optimizer

In [22]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)


Loss Function 정의

In [23]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


Time Step 정의

In [24]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns


번역 생성 함수

In [25]:
# 번역 생성 함수
def evaluate(sentence, model, src_tokenizer, tgt_tokenizer, max_len=50):
    sentence = preprocess_sentence(sentence)  # 전처리 함수 사용
    tokens = src_tokenizer.texts_to_sequences([sentence])[0]  # 토큰화

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens], maxlen=max_len, padding='post')

    output = tf.expand_dims([tgt_tokenizer.word_index['<start>']], 0)
    ids = []

    for i in range(max_len):
        predictions, _, _, _ = model(_input, output, None, None, None)
        predicted_id = tf.argmax(predictions[0, -1]).numpy()

        if predicted_id == tgt_tokenizer.word_index['<end>']:
            break
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    return ' '.join([tgt_tokenizer.index_word[id] for id in ids])

🏃🏃🏃💨 훈련 시작!! 

In [26]:
import tensorflow as tf
from tqdm import tqdm

# 하이퍼파라미터 설정
EPOCHS = 10
BATCH_SIZE = 64
BUFFER_SIZE = len(final_questions)
vocab_size = len(tokenizer.word_index) + 1  # 단어 사전 크기

# 1. Dataset 생성
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# 모델 및 옵티마이저 초기화
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=vocab_size,
    tgt_vocab_size=vocab_size,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True
)
learning_rate = LearningRateScheduler(512, warmup_steps=1000)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

# 2. 훈련 루프
for epoch in range(EPOCHS):
    total_loss = 0
    dataset_count = tf.data.experimental.cardinality(dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count, desc=f'Epoch {epoch + 1}', unit='batch')

    for (batch, (src, tgt)) in enumerate(dataset):
        loss, enc_attns, dec_attns, dec_enc_attns = train_step(src, tgt, transformer, optimizer)
        total_loss += loss
        tqdm_bar.set_postfix({'Loss': total_loss.numpy() / (batch + 1)})
        tqdm_bar.update(1)
    
    tqdm_bar.close()
    print(f'Epoch {epoch + 1} Loss: {total_loss.numpy() / dataset_count:.4f}')

# 예문 번역 생성
test_questions = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

for question in test_questions:
    print(f"\n질문: {question}")
    predicted_answer = evaluate(question, transformer, tokenizer, tokenizer)
    print(f"예측 답변: {predicted_answer}")

print("🫡 훈련 및 번역 생성 완료!")

Epoch 1: 100%|██████████| 738/738 [02:15<00:00,  5.44batch/s, Loss=4.95]


Epoch 1 Loss: 4.9479


Epoch 2: 100%|██████████| 738/738 [02:19<00:00,  5.28batch/s, Loss=2.19]


Epoch 2 Loss: 2.1867


Epoch 3: 100%|██████████| 738/738 [02:23<00:00,  5.14batch/s, Loss=1.21]


...

Epoch 8 Loss: 0.4926


Epoch 9: 100%|██████████| 738/738 [02:23<00:00,  5.15batch/s, Loss=0.46] 


Epoch 9 Loss: 0.4604


Epoch 10: 100%|██████████| 738/738 [02:23<00:00,  5.15batch/s, Loss=0.429]


Epoch 10 Loss: 0.4287

질문: 지루하다, 놀러가고 싶어.
예측 답변: 좋은 사람 만날 수 있을 거예요.

질문: 오늘 일찍 일어났더니 피곤하다.
예측 답변: 잘못 주무셨나봐요.

질문: 간만에 여자친구랑 데이트 하기로 했어.
예측 답변: 말하기 힘들었겠어요.

질문: 집에 있는다는 소리야.
예측 답변: 좋은 곳으로 데려다 깁 거예요.
🫡 훈련 및 번역 생성 완료!


## Step 7. 성능 측정하기
올바른 대답을 하는지 중요한 것이 평가 지표이다.

주어진 질문에 적절한 답변을 하는지 확인하는 'BLUE Score'을 계산

### 오류 🚨🚨🚨 ❓❓❓❓
BLUE 점수를 구현하려는데, 스코어 점수가 0점으로 나오는 걸 보면 아예 점수를 내지 못하는데.

그러한 이유를 분석해봤다.

`챗봇`의 경우 질문에 대한 단 하나의 정답이 존재하지 않기 때문에 BLEU Score만으로는 완벽하게 챗봇 응답의 질을 평가할 수 없다. 

하지만, 챗봇 답변이 참조 응답과 얼마나 유사한 표현을 사용하는지를 평가하는 용도로 BLEU Score를 `참고`할 수 있다고 한다.

In [33]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tqdm import tqdm

# BLEU Score 계산 함수 정의
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):

    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu([reference], candidate, weights=weights, smoothing_function=smoothie)
    return bleu_score

# 개별 문장에 대한 BLEU Score 계산 함수
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, verbose=True):

    # 모델을 사용하여 예측 답변 생성
    predicted_answer = evaluate(src_sentence, model, src_tokenizer, tgt_tokenizer)
    
    # 참조 문장과 예측 문장 토큰화
    reference = tgt_sentence.split()
    candidate = predicted_answer.split()
    
    # BLEU Score 계산
    bleu_score = calculate_bleu(reference, candidate)
    if verbose:
        print(f"Reference: {' '.join(reference)}")
        print(f"Candidate: {' '.join(candidate)}")
        print(f"BLEU Score: {bleu_score:.4f}")
    
    return bleu_score

# 전체 데이터셋에 대한 BLEU Score 평가 함수
def eval_bleu(model, src_sentences, tgt_sentences, src_tokenizer, tgt_tokenizer, verbose=False):

    total_score = 0.0
    sample_size = len(src_sentences)
    
    for idx in tqdm(range(sample_size), desc="Evaluating BLEU"):
        score = eval_bleu_single(model, src_sentences[idx], tgt_sentences[idx], src_tokenizer, tgt_tokenizer, verbose)
        total_score += score
    
    # 평균 BLEU Score 계산
    avg_bleu_score = total_score / sample_size
    print(f"\nNum of Samples: {sample_size}")
    print(f"Average BLEU Score: {avg_bleu_score:.4f}")
    return avg_bleu_score

# 예문 및 참조 답변 (테스트용)
test_questions = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

# 예시 답변 (BLEU 계산 시 참조 답변으로 사용)
reference_answers = [
    "<start> 잠깐 쉬 어도 돼요 . <end>",
    "<start> 맛난 거 드세요 . <end>",
    "<start> 떨리 겠 죠 . <end>",
    "<start> 좋 아 하 면 그럴 수 있 어요 . <end>"
]

# BLEU 평가 수행
avg_bleu_score = eval_bleu(transformer, test_questions, reference_answers, tokenizer, tokenizer, verbose=True)
print(f"Average BLEU Score: {avg_bleu_score:.4f}")

Evaluating BLEU:  50%|█████     | 2/4 [00:00<00:00,  4.36it/s]

Reference: <start> 잠깐 쉬 어도 돼요 . <end>
Candidate: 좋은 사람 만날 수 있을 거예요.
BLEU Score: 0.0000
Reference: <start> 맛난 거 드세요 . <end>
Candidate: 잘못 주무셨나봐요.
BLEU Score: 0.0000


Evaluating BLEU:  75%|███████▌  | 3/4 [00:00<00:00,  5.15it/s]

Reference: <start> 떨리 겠 죠 . <end>
Candidate: 말하기 힘들었겠어요.
BLEU Score: 0.0000


Evaluating BLEU: 100%|██████████| 4/4 [00:00<00:00,  4.17it/s]

Reference: <start> 좋 아 하 면 그럴 수 있 어요 . <end>
Candidate: 좋은 곳으로 데려다 깁 거예요.
BLEU Score: 0.0000

Num of Samples: 4
Average BLEU Score: 0.0000
Average BLEU Score: 0.0000


BLUE Score 구현을 잘못.. 했나?

당장 시험해보자

In [36]:
# 예문 및 참조 답변 (테스트용)
test_questions = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

# 예시 답변 (BLEU 계산 시 참조 답변으로 사용)
reference_answers = [
    "<start> 좋은 사람 만나 <end>",
    "<start> 잘못 주무셨나요 <end>",
    "<start> 말하기 힘들었지만 괜찮아요 <end>",
    "<start> 좋은 곳으로 데려갈거예요 <end>"
]

# BLEU 평가 수행
avg_bleu_score = eval_bleu(transformer, test_questions, reference_answers, tokenizer, tokenizer, verbose=True)
print(f"Average BLEU Score: {avg_bleu_score:.4f}")

Evaluating BLEU:  50%|█████     | 2/4 [00:00<00:00,  4.23it/s]

Reference: <start> 좋은 사람 만나 <end>
Candidate: 좋은 사람 만날 수 있을 거예요.
BLEU Score: 0.0972
Reference: <start> 잘못 주무셨나요 <end>
Candidate: 잘못 주무셨나봐요.
BLEU Score: 0.0248


Evaluating BLEU:  75%|███████▌  | 3/4 [00:00<00:00,  5.08it/s]

Reference: <start> 말하기 힘들었지만 괜찮아요 <end>
Candidate: 말하기 힘들었겠어요.
BLEU Score: 0.0151


Evaluating BLEU: 100%|██████████| 4/4 [00:00<00:00,  4.04it/s]

Reference: <start> 좋은 곳으로 데려갈거예요 <end>
Candidate: 좋은 곳으로 데려다 깁 거예요.
BLEU Score: 0.1212

Num of Samples: 4
Average BLEU Score: 0.0646
Average BLEU Score: 0.0646


예시 답변을 인위적으로 답변과 비슷하게 구현해봤는데 Score가 구해지는 것은 정상이였다 ㅎ

그저 성능이 안좋았을 뿐... 그래도 챗봇인지라 챗봇답게 구현해보겠다.

### 민혁의 챗봇

In [38]:
# 사용자 입력을 받아 챗봇 답변을 생성하는 함수
def chatbot_response(input_text, model, src_tokenizer, tgt_tokenizer):
    # 입력 문장 전처리
    processed_input = preprocess_sentence(input_text)
    
    # 모델 예측을 위한 번역 함수 호출
    response = evaluate(processed_input, model, src_tokenizer, tgt_tokenizer)
    
    return response

# 챗봇 인터페이스 시작
print("안녕하세요 민혁의 챗봇입니다. 종료하려면 '종료'라고 입력하세요.")
while True:
    user_input = input("당신: ")
    
    if user_input.lower() == "종료":
        print("챗봇: 대화를 종료합니다. 좋은 하루 보내세요!")
        break

    # 모델을 통한 답변 생성
    bot_response = chatbot_response(user_input, transformer, tokenizer, tokenizer)
    print(f"챗봇: {bot_response}")

안녕하세요 민혁의 챗봇입니다. 종료하려면 '종료'라고 입력하세요.
당신: 오늘 점심을 추천해줄래?
챗봇: 쉽지 않은 결정이었을텐데 슬픔 갈증 많았어요.
당신: 나 너무 배고파 뭘 먹을까?
챗봇: 좀 더 마음이 아프네요.
당신: 무엇을 먹을지 추천해줘
챗봇: 끝을 아는 것도 중요한 것 같아요.
당신: 어제 저녁을 안 먹었더니. 너무 허기져
챗봇: 슬픈 예감은 틀린 적이 없죠.
당신: 지금 과자 예감을 먹으라고 한거야?
챗봇: 챙겨주고 싶나봐요.
당신: 너 먹을 줄 아는구나?
챗봇: 사람은 힘이 되지 않겠지만 힘내세요.
당신: 이제 그만하자
챗봇: 좋은 시간이 만날 수 있을 거예요.
당신: 종료
챗봇: 대화를 종료합니다. 좋은 하루 보내세요!


## 회고
이번 학습과 프로젝트에서 번역과 관련된 데이터, 모델, 성능 평가 등 많은 것들을 배워보며 실습을 진행해봤다.

실습을 했던 과정을 프로젝트로 적용해보니 쉽지 않았지만.

재밌었고 또한 GPT 같은 모델은 얼마나 많은 학습을 했고 많은 노력으로 만든 챗봇인지 체감이 더 되었다. 함수를 구현하는 부분이라던지 더 코드적인 면에서 모듈화하고 함수화해서 더 공부하자는 생각이 들었다.